In [4]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from tensorflow.keras.callbacks import TensorBoard
# from sklearn.model_selection import train_test_split


from tensorflow.keras.models import load_model
import numpy as np
import cv2
import mediapipe as mp

from IPython.display import clear_output
import tensorflow as tf




In [ ]:
# DATA COLLECTING
import cv2
import mediapipe as mp
from IPython.display import clear_output

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# cap = cv2.VideoCapture("vid4.mp4")
cap = cv2.VideoCapture(0)


# PARAMETERS:
# ================================================================================================================================
# timer to ready before posing/getting ready for the excercise
set_timer_start = 50

# ================================================================================================================================
# range of how much change will it be considered a movement
# lower -> small movement and jittery flaw of mediapipe will be detected
# higher -> it wont detect immediately important frames
# 0-1
change_range = .03

# ================================================================================================================================
# a counter for the amount of time for the wait time before performing the exercise again
between_exercise_ctr = 6



isInitiated = False
exercise_exectuted_ctr=0
prev_x = 0
prev_y = 0
no_movement_ctr = 0
movement_boolean = False
timer = 0
no_movement_threshold = 32

prev_array = []

for x in range(33):
    prev_array.append([0, 0])

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if not ret:
        break

    timer += 1 
    landmark_all_present = False

    if timer == set_timer_start:
        file = open('base1234.txt', 'w')
        if isInitiated==False:
            file.write("START")
            file.write('\n')
        isInitiated = True

    if results.pose_landmarks:
        ctr_check=0

        for idx, landmark in enumerate(results.pose_landmarks.landmark):
            if idx == 32:
                landmark_all_present = "Landmarks -> ALL PRESENT"
            else:
                landmark_all_present = "Landmarks -> SOME ARE MISSING"

            x = landmark.x
            y = landmark.y

#checks if no movement is detected-----------------------------------------------------------------------------------------------------------------------------------------
            if x >= (prev_array[idx][0] - change_range) and y <= (prev_array[idx][1] + change_range) and isInitiated:   
                print(idx,")",x,"---",prev_array[idx][0],'---> NO MOVEMENT')
                ctr_check+=1          
                
            else:
                print(idx,")",x,"---",prev_array[idx][0],'--->YES MOVEMENT',)
                movement_boolean = True
                no_movement_ctr = 0

            if ctr_check == no_movement_threshold:
                no_movement_ctr += 1 

            if no_movement_ctr>=25:
                movement_boolean=False
            prev_array[idx] = [x, y]
        print('counter->',no_movement_ctr)
        print(movement_boolean)

#if movement is detected then it will record the coordinates into a text file-----------------------------------------------------------------------------------------------------
        if movement_boolean:
            text = "MOVEMENT"      
            if timer >= set_timer_start and file is not None:
                for landmark in prev_array:
                    # file.write("<")
                    file.write("|")
                    file.write(str(landmark[0]))
                    file.write("|")
                    file.write(str(landmark[1]))
                    # file.write(">")
                    # file.write("|")
                file.write('\n')

        if no_movement_ctr >= 5:    
            movement_boolean = False
            text = "NO MOVEMENT!"

        if no_movement_ctr == between_exercise_ctr:
            exercise_exectuted_ctr+=1
            file.write("END")  
            file.write('\n')
            file.write('\n')
            file.write("START")
            file.write('\n')



        if timer >= set_timer_start and no_movement_ctr >=between_exercise_ctr:
            cv2.putText(frame, "PERFORM THE EXERCISE", (180, 460), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)
# -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


#Outputing important informations-----------------------------------------------------------------------------------------------------------------------------------------------
        cv2.putText(frame, str(movement_boolean), (20, 20), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
        cv2.putText(frame, f"no_movement_ctr-> {no_movement_ctr}", (20, 37), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
        cv2.putText(frame, str(landmark_all_present), (20, 54), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
        cv2.putText(frame, f"timer-> {timer}", (20, 73), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
        cv2.putText(frame, f"exercise exectuted-> {exercise_exectuted_ctr}", (20, 92), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
# -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        if timer < set_timer_start:
            cv2.putText(frame, "PREPARE YOUR STARTING POSE", (180, 460), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)
            

        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
        )

    cv2.imshow('MediaPipe Pose', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

if file is not None:
    file.close()
cap.release()
cv2.destroyAllWindows()


In [ ]:
import numpy as np

def txt_pre_process(txt_file,label,simplify=False,simplify_level=4 ):
    label_array = []
    temp_feature_data = []
    temp_sequence_data = []
    batch_data = []

    with open(str(txt_file), 'r') as file:

        for line in file:
            values = line.strip().split('|')        

            temp_feature_data = []

            for value in values:            
                float_value = str(value)

                #FIRST PART OF THE SEQUENCE 
                if float_value == 'START':   
                    temp_sequence_data=[]
                                        
                elif float_value == 'END':              
                    batch_data.append(temp_sequence_data)
                    label_array.append(label)

            
                elif float_value != '' and float_value != 'START': 
                    if simplify:    
                        float_value = round(float(value),simplify_level)
                    else:
                        float_value = float(value)
                    temp_feature_data.append(float_value)

            if temp_feature_data!=[]:
                temp_sequence_data.append(temp_feature_data)

    label_array = np.array(label_array)
    return [batch_data,label_array]

#--------------------------------------------------------------------------- padding --------------------------------------------------------------------------------
# padding can be improved probably...by using sequence 
# minor issue:
# > is whether sequences had exceeded the intended number of sequences but is still right (it was performed right but slower(by an acceptable margin)) - not resolved
#    = temporary fix was just to truncate everything if it had exceeded the intended number of sequence for the sake of running it for now
#    = a reliable solution in theory could be that to randomly truncate in between the first and end sequence, in this way relevant data can be captured


def padding(pre_processed_input,optional_maxLength=0): 
    padded_sequences = []
    if optional_maxLength != 0:
        max_length = optional_maxLength
    else:
        max_length = max(len(sequence) for sequence in pre_processed_input)
    
    for sequence in pre_processed_input:        
        padding_length = max_length - len(sequence)
        if padding_length >= 0:
            padded_sequence = np.pad(sequence, ((0, padding_length), (0, 0)), mode='constant')
            
        else:
            padded_sequence = sequence[:max_length]
        padded_sequences.append(padded_sequence)
    padded_sequences = np.array(padded_sequences)

    return padded_sequences

#--------------------------------------------------------------------------- padding --------------------------------------------------------------------------------

# this is to merge correct executions and wrong executions and randomize their input and label
# positions of input and its corresponding label are the same 
def concatenate_randomize_batches(base_input,base_label,concat_input,concat_label):
    combined_inputs = np.concatenate((base_input,concat_input), axis = 0)
    combined_label = np.concatenate((base_label,concat_label), axis = 0)
    indices = np.random.permutation(len(combined_inputs))
    randomized_inputs = combined_inputs[indices]
    randomized_label = combined_label[indices]
    return [randomized_inputs,randomized_label]


def predict_sequence(sequence_set,model,optional_maxLength=0):
    loaded_model = tf.keras.models.load_model(model)





In [ ]:
temp_correct = txt_pre_process('base(correct_execution).txt',1,True,4)
temp_wrong = txt_pre_process('base(wrong_execution).txt',0,True,4)

base_input = padding(temp_correct[0])
concat_input = padding(temp_wrong[0],31)

base_label = temp_correct[1]
concat_label = temp_wrong[1]

# optional if labels were not initially assigned
# correct_labels = np.ones(len(base_input))  # Assign label 1 to correct inputs
# wrong_labels = np.zeros(len(concat_input))  # Assign label 0 to wrong inputs

rand_batches=concatenate_randomize_batches(base_input,base_label,concat_input,concat_label)

print("-------------------checking:-------------------")
print("base_input->",base_input.shape)
print("concat_input->",concat_input.shape)
print("successfully merged and randomized(input) ->",rand_batches[0].shape)
print("successfully merged and randomized(label) ->",rand_batches[1].shape)

In [ ]:
# version 2 - included a dropouts
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', dropout=0.1, recurrent_dropout=0.1, input_shape=(31, 66)))
model.add(LSTM(128, return_sequences=True, activation='relu')) 
model.add(LSTM(64, return_sequences=False, activation='relu')) 
model.add(Dense(64, activation='relu')) 
model.add(Dense(32, activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer = 'Adam' , loss = 'binary_crossentropy', metrics=['accuracy', 'binary_accuracy'])



# splits data into train and test set

X_train, X_test, y_train, y_test = train_test_split(rand_batches[0], rand_batches[1], test_size=0.1, random_state=42)

# further splitting the dataset to have a set for simulation later on
X_train, predict_simulate_input, y_train, predict_simulate_label = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)



model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))


In [ ]:
run_model = tf.function(lambda x: model(x))
# This is important, let's fix the input size.
BATCH_SIZE = 1
STEPS = 31
INPUT_SIZE = 66
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], model.inputs[0].dtype))

# model directory.
MODEL_DIR = "keras_lstm"
model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
tflite_model = converter.convert()

In [ ]:
# Run the model with TensorFlow to get expected results.
TEST_CASES = 10

# Run the model with TensorFlow Lite
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

for i in range(TEST_CASES):
  expected = model.predict(X_test[i:i+1])
  interpreter.set_tensor(input_details[0]["index"], X_test[i:i+1, :, :])
  interpreter.invoke()
  result = interpreter.get_tensor(output_details[0]["index"])

  # Assert if the result of TFLite model is consistent with the TF model.
  np.testing.assert_almost_equal(expected, result, decimal=5)
  print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

  # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
  # the states.
  # Clean up internal states.
  interpreter.reset_all_variables()

In [ ]:
# sample_index = 0
# sample_input = X_test[sample_index]

# # Set the input tensor to the input data
# interpreter.set_tensor(input_details[0]['index'], [sample_input])

# # Perform inference
# interpreter.invoke()

# # Get the output tensor
# output_data = interpreter.get_tensor(output_details[0]['index'])

# # Process the output to get the predicted class
# predicted_class = np.argmax(output_data)
# print("Predicted Class:", predicted_class)
# print("Predicted Class:", output_data)

In [69]:

prediction =[]
one_exercise = []
# max length of sequence
for y in range(31):
    sequence_input=[]
    # max num of landmark x and y flattened
    for x in range(66):
        sequence_input.append(0)
    one_exercise.append(sequence_input)


mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# cap = cv2.VideoCapture("vid4.mp4")
cap = cv2.VideoCapture(0)


while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if not ret:
        break


    if results.pose_landmarks:
        temp_sequence=[]
        for idx, landmark in enumerate(results.pose_landmarks.landmark):
            x = landmark.x
            y = landmark.y
            # print(idx,")",x,"---",prev_array[idx][0],)
            # print(idx)
            temp_sequence.append(x)
            temp_sequence.append(y)

    # if max length of the exercise sequence was reached then pop it pop off
        if len(one_exercise) >= 31:
            one_exercise.pop(0)


        # one_exercise.append(temp_sequence)
        # prediction = np.array(one_exercise)
        # print(len(one_exercise))
        # print(prediction.shape)
        # prediction = prediction.reshape(1,31,66)
        # print(model.predict(prediction))

    mp_drawing.draw_landmarks(
        frame,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
    )

    cv2.imshow('MediaPipe Pose', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [3]:
interpreter = tf.lite.Interpreter(model_path='converted_model.tflite')
interpreter.allocate_tensors()